#### 2-1. Nate 뉴스기사 제목 스크래핑하기
https://news.nate.com/recent?mid=n0100
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 
6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기

Image, 기사제목, 기사링크


□ 뉴스기사의 Image를 출력 하세요 
1) Image의 도메인이름이 포함된 url과 src 속성의 img 경로를 합치려면 urljoin 함수를 사용하세요.
    from urllib.parse import urljoin
    
    url = ‘https://news.nate.com/recent?mid=n0100’
    src=’ //thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg’

2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
    from IPython.display import Image, display

3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
  => Image 가 없는 뉴스도 있기 때문에


#### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기
 하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다
  ex) def download_one_episode(title,no,url):

#### 2-3. 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기
하나의 웹툰의 제목(title)과 회차를 알 수 있는 url을 입력으로 받는 함수를 선언합니다. ex) def download_all_episode(title, episode_url):